In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [2]:
servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)
navegador.maximize_window()

In [3]:
def busca_google_shopping(navegador,produto,termos_banidos,preco_minimo,preco_maximo):
    #entrar no google
    navegador.get('https://www.google.com/')
    
    #tratar valores inseridos
    produto = produto.lower()
    lista_termos_produtos = produto.split(" ")
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    
    # adicionar o produto na busca do google
    navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(produto)
    navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
    
    #procura pelo shopping google
    navegador.find_element(By.XPATH,'//*[@id="hdtb-msb"]/div[1]/div/div[2]/a').click()
        
    #pega lista de resultados do google shopping
    lista_resultados = navegador.find_elements(By.CLASS_NAME, 'sh-dgr__grid-result')
    lista_ofertas =[]
    
    for resultado in lista_resultados:
        preco = resultado.find_element(By.CLASS_NAME,'XrAfOe').text
        preco = preco.split('\n')
        preco = preco[1]
        try:
            preco = preco.replace('R$','').replace(' ','').replace('.','').replace(',','.')
            preco = float(preco)
        except:
            continue
        
        nome = resultado.find_element(By.CLASS_NAME,'tAxDx').text
        nome = nome.lower()
        
        #verificando o nome 
        tem_termos_banidos = False
        for palavra in lista_termos_banidos:
            if palavra in nome:
                tem_termos_banidos =True
                
        tem_todos_termos_produto = True
        for palavra in lista_termos_produtos:
            if palavra not in nome:
                tem_todos_termos_produto = False
                
        # se tem_temosPbanidos = False e tem_todos_termos_produtos = True -- executa a analise
        preco_maximo = float(preco_maximo)
        preco_minimo = float(preco_minimo)
        if preco_minimo <= preco <= preco_maximo:
            elemento_link = resultado.find_element(By.CLASS_NAME,'aULzUe')
            elemento_pai = elemento_link.find_element(By.XPATH,'..')
            link = elemento_pai.get_attribute('href')
            lista_ofertas.append((nome,preco,link))
    return lista_ofertas

In [34]:
produto = 'Smartphone Samsung Galaxy'
termos_banidos = 'A53'
preco_minimo = 3000
preco_maximo = 3500

lista_produtos = busca_google_shopping(navegador,produto,termos_banidos,preco_minimo,preco_maximo)

In [27]:
print(lista_produtos)

[('smartphone samsung galaxy a32, 128gb, 4gb ram, tela 6.4" - violeta', 1399.0, 'https://www.google.com/url?url=https://www.americanas.com.br/produto/2969968331%3Fopn%3DYSMESP&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwig2rmbxK78AhWbF7kGHfZmDcEQ_uQECJoQ&usg=AOvVaw0_llRuzjpJu-btoqYkeQzm'), ('smartphone samsung galaxy m23 5g 128gb 6gb ram verde', 1349.1, 'https://www.google.com/url?url=https://www.magazineluiza.com.br/smartphone-samsung-galaxy-m23-128gb-verde-5g-octa-core-6gb-ram-66-cam-tripla-selfie-8mp/p/234879200/te/galx/%3F%26utm_source%3Dgoogle%26utm_medium%3Dsmart_pla%26utm_campaign%3Dgoop_samsung_cadastra_smart_pla%26partner_id%3D65686%26seller_id%3Dmagazineluiza&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwig2rmbxK78AhWbF7kGHfZmDcEQguUECKoQ&usg=AOvVaw1WjtVOyFxRgdf7g8BaDX7N'), ('smartphone samsung galaxy a13 128gb azul 4gb ram', 989.1, 'https://www.google.com/url?url=https://www.casaevideo.com.br/smartphone-samsung-a135-galaxy-a13-128gb-azul/p%3Fidsku%3D190996%26srsltid%3DAeTuncoO16berA1rqvfro4j9n0Oy7wCQ

In [8]:
def busca_buscape(navegador, produto, termos_banidos, preco_minimo, preco_maximo):
    # tratar os valores da função
    preco_maximo = float(preco_maximo)
    preco_minimo = float(preco_minimo)
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_produto = produto.split(" ")
    
    
    # entrar no buscape
    navegador.get("https://www.buscape.com.br/")
    
    # pesquisar pelo produto no buscape
    navegador.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto, Keys.ENTER)
    
    # pegar a lista de resultados da busca do buscape
    time.sleep(5)
    lista_resultados = navegador.find_elements(By.CLASS_NAME, 'SearchCard_ProductCard_Inner__7JhKb')
    
    # para cada resultado
    lista_ofertas = []
    for resultado in lista_resultados:
        try:
            preco = resultado.find_element(By.CLASS_NAME, 'Text_Text__h_AF6.Text_MobileHeadingS__Zxam2').text
            nome = resultado.find_element(By.CLASS_NAME,'Text_Text__h_AF6.Text_MobileLabelXs__ER_cD.Text_DesktopLabelSAtLarge__baj_g.SearchCard_ProductCard_Name__ZaO5o').text
            nome = nome.lower()
            link = resultado.get_attribute('href')
            
            # verificacao do nome - se no nome tem algum termo banido
            tem_termos_banidos = False
            for palavra in lista_termos_banidos:
                if palavra in nome:
                    tem_termos_banidos = True  
                    
            # verificar se no nome tem todos os termos do nome do produto
            tem_todos_termos_produto = True
            for palavra in lista_termos_produto:
                if palavra not in nome:
                    tem_todos_termos_produto = False            
            
            if not tem_termos_banidos and tem_todos_termos_produto:
                preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
                preco = float(preco)
                if preco_minimo <= preco <= preco_maximo:
                    lista_ofertas.append((nome, preco, link))
        except:
            pass
    return lista_ofertas

In [36]:
produto = 'iphone 11 64 gb'
termos_banidos = 'mini watch'
preco_minimo = 2000
preco_maximo = 12000

lista_ofertas_busca = busca_buscape(navegador, produto, termos_banidos, preco_minimo, preco_maximo)

In [31]:
print(lista_ofertas_busca)

[('smartphone apple iphone 11 64gb câmera dupla', 2930.0, 'https://www.buscape.com.br/celular/smartphone-apple-iphone-11-64gb-ios?_lc=88&searchterm=iphone%2011%2064%20gb'), ('smartphone apple iphone 11 vermelho 64gb câmera dupla', 2998.45, 'https://www.buscape.com.br/celular/smartphone-apple-iphone-11-vermelho-64gb-ios?_lc=88&searchterm=iphone%2011%2064%20gb'), ('smartphone apple iphone 11 pro max 64gb câmera tripla', 4297.0, 'https://www.buscape.com.br/celular/smartphone-apple-iphone-11-pro-max-64gb-ios?_lc=88&searchterm=iphone%2011%2064%20gb'), ('smartphone apple iphone 11 usado 64gb câmera dupla', 2559.92, 'https://www.buscape.com.br/celular/smartphone-apple-iphone-11-usado-64gb-ios?_lc=88&searchterm=iphone%2011%2064%20gb'), ('smartphone apple iphone 11 pro max usado 64gb câmera tripla', 3800.72, 'https://www.buscape.com.br/celular/smartphone-apple-iphone-11-pro-max-usado-64gb-ios?_lc=88&searchterm=iphone%2011%2064%20gb'), ('smartphone apple iphone 11 pro usado 64gb câmera tripla', 

In [5]:
tabela = pd.read_excel('buscas.xlsx')
display(tabela)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,Smartphone Samsung Galaxy,A53,1500,3500
1,rtx 3060,zota galax,4000,4500


In [10]:
tabela_ofertas = pd.DataFrame()

for linha in tabela.index:
    produto = tabela.loc[linha,'Nome']
    termos_banidos = tabela.loc[linha,'Termos banidos']
    preco_minimo = tabela.loc[linha,'Preço mínimo']
    preco_maximo = tabela.loc[linha,'Preço máximo']
    
    lista_ofertas_google_shopping = busca_google_shopping(navegador,produto,termos_banidos,preco_minimo,preco_maximo)
    if lista_ofertas_google_shopping:
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns=['nome', 'preco', 'link'])
        tabela_ofertas = tabela_ofertas.append(tabela_google_shopping)
    else:
        tabela_google_shopping = None
        
    lista_ofertas_buscape = busca_buscape(navegador, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=['nome', 'preco', 'link'])
        tabela_ofertas = tabela_ofertas.append(tabela_buscape)
    else:
        tabela_buscape = None

display(tabela_ofertas)

,nome,preco,link
0,"smartphone samsung galaxy a32, 128gb, 4gb ram,...",1709.10,https://www.google.com/url?url=https://www.fas...
1,"smartphone samsung galaxy a73 128gb 5g, câmera...",1599.00,https://www.google.com/url?url=https://lojaonl...
2,smartphone samsung galaxy s21 fe 5g 128gb 6gb ...,2599.00,https://www.google.com/url?url=https://www.com...
3,"smartphone samsung galaxy s20 fe 5g, 128gb, 6g...",1899.05,https://www.google.com/url?url=https://www.cea...
4,smartphone samsung galaxy a32 128gb azul,1568.00,https://www.google.com/url?url=https://www.nov...
5,smartphone samsung galaxy a13 6.6 polegadas oc...,1541.30,https://www.google.com/url?url=https://www.cas...
6,smartphone samsung galaxy s21 fe 5g verde 128g...,2799.00,https://www.google.com/url?url=https://www.bem...
7,smartphone samsung galaxy a22 128gb 4gb ram - ...,1529.10,https://www.google.com/url?url=https://www.web...
8,smartphone galaxy s20 fe 5g 128gb 6gb violeta ...,2249.10,https://www.google.com/url?url=http://www.nage...
9,smartphone samsung galaxy a53 5g 128gb branco ...,1776.67,https://www.google.com/url?url=https://www.ons...


In [ ]:
# exportar pro excel
tabela_ofertas = tabela_ofertas.reset_index(drop=True)
tabela_ofertas.to_excel("Ofertas.xlsx", index=False)


In [ ]:
if len(tabela_ofertas) > 0:
    #enviando o email
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = 'xxxxxxxx@gmail.com'
    mail.Subject = 'Produto(s) Encontrado(s) na faixa de preço desejada'
    mail.HTMLBody = f"""
    <p>Ola,</p>
    <p>Encontramos estes produtos em oferta dentro da faixa de preço desejada. Segue tabela com detalhes</p>
    {tabela_ofertas.to_html(index=False)}
    <p>Qualquer dúvida estou à disposição</p>
    <p>Att.,</p>
    """
    
    mail.Send()

nav.quit()  